In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import FloatType

In [7]:
spark = SparkSession.builder.appName("L3Q2").getOrCreate()

In [19]:
df1 = spark.read.csv('datasets/Tokenized.csv/part-00000-8ad89596-b847-43fc-b712-ec12acf2c8ce-c000.csv', header=True)
df2 = spark.read.csv('datasets/Tokenized.csv/part-00001-8ad89596-b847-43fc-b712-ec12acf2c8ce-c000.csv', header=True)

df = df1.union(df2)


In [20]:
df = df.withColumn('tokens', split('tokens', ','))
df_pairs = df.alias("a").crossJoin(df.alias("b")).where(col("a.id") < col("b.id"))

In [32]:
def jaccard_similarity(tokens1, tokens2):
    set1 = set(str(tokens1))
    set2 = set(str(tokens2))
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0
    return float(len(intersection)) / len(union)

In [33]:
jaccard_udf = udf(jaccard_similarity)

In [35]:
result = df_pairs.withColumn("jaccard_similarity",jaccard_udf(col("a.tokens"), col("b.tokens"))).select(col('a.id').alias('id_1'), col('b.id').alias('id_2'),col('jaccard_similarity'))
result.show(5)

+----+----+-------------------+
|id_1|id_2| jaccard_similarity|
+----+----+-------------------+
|1859|3503| 0.5555555555555556|
|1859| 276|0.42105263157894735|
|1859|4600|0.23809523809523808|
|1859| 580|               0.35|
|1859|4861|0.42105263157894735|
+----+----+-------------------+
only showing top 5 rows
